# Course1-Project1: EdTech Backend System

In [23]:
import hashlib
from openpyxl import load_workbook
#from enum import Enum
#class Role(Enum):
#    LEARNER = "Learner"
#    INSTRUCTOR = "Instructor"

file_path = r"C:\Users\ecarald\OneDrive - Ericsson\Desktop\PG DE\Course 1\instructors.xlsx"

def set_id(name):
    # Encode the string and compute a hash (SHA-256)
    hash_name = hashlib.sha256(name.encode('utf-8'))
    # Convert the hexadecimal hash to an integer
    id = int(hash_name.hexdigest(), 16)
    # Take only part of it (last 10 digits)
    return id % (10**10)
        
class User:
    counter = 0
    
    def __init__(self, name, role=None, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'learner' or 'instructor'
        'email' and 'password' are optional.
        '''
        #if not isinstance(role,Role):
        #    raise ValueError("role must be a member of Role Enum.")
            
        self.name = name
        self.id=set_id(name)
        self.role = role
        self.email = email
 
        if password:
            self.hash_password(password)
            self.dotify_password(password)

        User.counter +=1
        
    def set_email(self, email):
        self.email=email

    def hash_password(self, password):
        '''
        Hash the password using SHA-256 before storing it
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed
        
    def dotify_password(self, password):
        '''
        Mask the password with dots
        '''
        self.dotified_password = '.' * len(password)

    def validate_password(self,password):
        """
        Check if a given password matches the stored hashed password.
        """
        if self.__password_hash is None:
            return False
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        return hashed == self.__password_hash
        
    def validate_email(self,email):
        return self.email == email            

    def set_role(self):
        option = int(input("Choose either (1) for 'Learner' or (2) for 'Instructor': "))
        if option == 1:
            self.role = "Learner"
        elif option == 2:
            self.role = "Instructor"
        else:
            raise ValueError("Wrong value. Role can only be either (1) for 'Learner' or (2) for 'Instructor")
        return self.role
        
    def print_info(self):
        print("Name    :", self.name)
        print("User Id :", self.id)
        print("Role    :", self.role)
        print("Email   :", self.email)
        print("Password:", self.dotified_password)


class Instructor(User):
    def __init__(self, name, email, taught_courses):
        User.__init__(self, name, "Instructor", email)
        self.taught_courses = taught_courses # 'taught_courses' is a simple list of all the courses that instructor teaches

    def print_info(self):
        print("Instructor name:",self.name)
        print("Instructor Id  :",self.id)
        print("Taught courses:")
        for course in self.taught_courses:
            print(course)

class Course:
    def __init__(self, name, instructors):
        '''Build course from instructors list'''
        self.name = name
        self.id = set_id(name)
        # instructors is a list of tuples: [(name, id), (name, id), ...]
        self.instructors = instructors

    def __str__(self):
        instructors_str = ", ".join(
            [f"{{{name} : {id_}}}" for name, id_ in self.instructors]
        )
        return f"Course name: {self.name}\nCourse ID: {self.id}\nInstructors: {instructors_str}"
   
    def print_info(self):
        print("Course name:",self.name)
        print("Course Id  :",self.id)
        print("List of instructors:")
        for instructor in self.instructors:
            print(instructor.name, "-", instructor.id)


class Backend():
    def __init__(self): 
        pass

    def load_instructors_from_excel(self, file_path):
        self.instructors = [] #'instructors' is a simple list of instances of class Instructor
        
        # Load workbook and select first sheet
        workbook = load_workbook(filename=file_path)
        sheet = workbook.active        

        # Skip the header row → start from row 2
        for row in sheet.iter_rows(min_row=2, values_only=True):
            name = row[0]
            surname = row[1]
            email = row[2]
            courses = [cell for cell in row[3:] if cell is not None]  # skip empty cells

            full_name = f"{name} {surname}"
            instructor = Instructor(full_name, email, courses)
            self.instructors.append(instructor)

    def generate_courses_from_instructors(self):
        '''
        Creates Course objects based on instructors' taught courses.
        Each Course has a name and a list of instructor names teaching it.
        '''
        courses_dict = {} #´course_name -> list of instructor names

        #Build mapping from course -> instructor list
        for instructor in self.instructors:
            for course_name in instructor.taught_courses:
                if course_name not in courses_dict:
                    courses_dict[course_name] = [] #crea una entrada con sólo la clave ´course-name'
                courses_dict[course_name].append((instructor.name, instructor.id)) # popula la entrada con el instructor name y id

        # Convert mapping into Course objects
        self.courses = [
            Course(name, instructors_list)
            for name, instructors_list in courses_dict.items()
        ]
    
    def print_all_instructors(self):
        if not self.instructors:
            print("No instructors found.")
            return
        for i in self.instructors:
            i.print_info()
            print("-"*30)
    
    def print_all_courses(self):
        '''Display info for all stored courses.'''
        if not self.courses:
            print("No courses found.")
            return
        for course in self.courses:
            course.print_info()
            print("-"*30)

backend = Backend()
backend.load_instructors_from_excel(file_path)
print("List of all available instructors")
backend.print_all_instructors()

backend.generate_courses_from_instructors()
print("\nList of all available courses")
for course in backend.courses:
    #print(course.name, "->", course.instructors)
    print(course)
    print("-" * 30)
#backend.print_all_courses()



List of all available instructors
Instructor name: Alice Aaa
Instructor Id  : 5535300755
Taught courses:
Algebra
Biology
Calculus
------------------------------
Instructor name: Bob Bbb
Instructor Id  : 6623670666
Taught courses:
Algebra
Calculus
Dentist
------------------------------

List of all available courses
Course name: Algebra
Course ID: 2872948788
Instructors: {Alice Aaa : 5535300755}, {Bob Bbb : 6623670666}
------------------------------
Course name: Biology
Course ID: 9230804429
Instructors: {Alice Aaa : 5535300755}
------------------------------
Course name: Calculus
Course ID: 3793446076
Instructors: {Alice Aaa : 5535300755}, {Bob Bbb : 6623670666}
------------------------------
Course name: Dentist
Course ID: 9540905890
Instructors: {Bob Bbb : 6623670666}
------------------------------
